# Building an AI Email Assistant with Mistral AI and Composio

This notebook demonstrates how to build an AI-powered email assistant using [Mistral AI](https://mistral.ai/) and [Composio](https://app.composio.dev/). By using Mistral’s large language models (LLMs) and Composio’s Gmail tool, we can create a system capable of performing common Gmail actions such as sending, fetching, drafting emails etc., all through natural language commands.

In this guide, we'll create an AI email assistant that can automatically reply to received emails! 

## Setup and Dependencies


### Python Libraries

Install the necessary libraries to set up the AI agent - 

In [ ]:
!pip install composio_langchain composio_core langchain-mistralai -U 

### Mistral AI API

Sign up at [Mistral AI](https://mistral.ai/) and generate an API key from the console. This key will be used to access Mistral's LLM models.

In [1]:
import os
os.environ['MISTRAL_API_KEY']="your_mistral_api_key"

### Composio set up

In [ ]:
!composio login

In your command line, run - 

In [ ]:
!composio add gmail

This will set up an integration between your gmail account and Composio.

Also, run - 

In [ ]:
!composio triggers enable gmail_new_gmail_message

This command enables the trigger for when a new mail is received.

## Building the Email Assistant

### Initialize Composio's tools

The first thing we do is initialize Composio's tools.

In [ ]:
from composio_langchain import Action, ComposioToolSet

toolset = ComposioToolSet()
tools = toolset.get_tools(
    actions=[
        Action.GMAIL_REPLY_TO_THREAD,
        Action.GMAIL_SEND_EMAIL,
    ],
)

The various actions represent functions that the LLM can call. In order for Mistral models to understand the functions, we need to outline the function specifications with a JSON schema. Specifically, we need to describe the type, function name, function description, function parameters, and the required parameter for the function. Composio does this and provides an optimized JSON schema, making it very easy to integrate external tools with LLMs. For example, for the `GMAIL_SEND_EMAIL` action, this is the schema provided to the model - 

```
{
	"properties":{
		"recipient_email":{
			"description":"Email address of the recipient",
			"examples":["john@doe.com"],
			"title":"Recipient Email",
			"type":"string"
			},
			"subject":{
				"description":"Subject of the email",
				"examples":["meeting"],
				"title":"Subject",
				"type":"string"
			},
			"body":{
				"description":"Body content of the email",
				"examples":["Hey, nice talking to you"],
				"title":"Body",
				"type":"string"
			},
			"user_id":{
				"default":"me",
				"description":"The user's email address or 'me' for the authenticated user.",
				"title":"User Id",
				"type":"string"
			}
		},
		"required":[
			"recipient_email",
			"subject",
			"body"
		],
		"title":"CreateEmailDraftRequest",
		"type":"object"
}

### Setting up the model



In [4]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatMistralAI(model="mistral-large-latest")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI email assistant that can write and reply to emails. Follow the user’s instructions carefully and perform the requested actions.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

We then create the agent and also create a listener for the trigger we created above.

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Create a trigger listener
listener = toolset.create_trigger_listener()

After this, we can attach a callback to the listener. This callback function is called whenever the trigger receives data.

In [ ]:
from composio import Trigger
@listener.callback(filters={"trigger_name": Trigger.GMAIL_NEW_GMAIL_MESSAGE})
def on_new_gmail_message(event) -> None:
    try:
        print("data received", event)

				# Extract the relevant information from the event
        payload = event.payload
        thread_id = payload["threadId"]
        message = payload["messageText"]
        sender_mail = payload["sender"]

        # you can also customize input to only respond to certain emails
        res = agent_executor.invoke({"input": f"This is the mail you have to respond to: {message}. It's from {sender_mail} and the threadId is {thread_id}. Generate a response based on the {message} and then reply through tool calling."})
        print(res)
    except Exception as e:
        print("Error:", e)

print("Listener started!")
listener.wait_forever()

As can be seen in the above code, there are a lot of customization possiblities for the agent, both in the information that can be extracted from the trigger and how that information is used. 

## Conclusion

In this guide, we built an AI email assistant by combining Mistral’s LLMs with Composio’s prebuilt actions. By following the steps outlined in this guide, you can extend this framework to other applications, creating versatile tools that leverage the power of LLMs and function calling.



### Connect with Composio and learn more

If you encounter any problems, please let us know at out [Discord](https://discord.com/invite/cNruWaAhQk).

Check out [Composio's documentation](https://docs.composio.dev/introduction/intro/overview) to learn more about how to use and integrate various tools for different usecases.